## 4 NSA und SQL (7 Punkte)
Copyright Jens Dittrich & Christian Schön, [Big Data Analytics Group](https://bigdata.uni-saarland.de/), [CC-BY-SA](https://creativecommons.org/licenses/by-sa/4.0/legalcode)

## Requirements

This notebook is based on the sqlite3-kernel by Andrew Brownan which is [available on GitHub](https://github.com/brownan/sqlite3-kernel). As the kernel is based on the bash shell, it will only run on Unix systems, but not on Windows.

The easiest way to use this notebook is to use our `Vagrantfile` as explained [here](https://github.com/BigDataAnalyticsGroup/vagrant-bde).

Alternatively, if you want to install the sqlite kernel yourself, follow these steps:
1. Download the repository as zip file or clone it using git, if necessary unpack the archive.
2. If you are using a virtual machine, copy the folder to a location accessible by the virtual machine, e.g. a shared folder. Run your virtual machine and if necessary start your python environment.
3. Move to the folder containing the kernel and execute the following commands:
  - python setup.py install
  - python -m sqlite3_kernel.install
  
If the kernel was successfully installed, you should now be able to start jupyter notebook and select "Sqlite3" as notebook type for new notebooks. 

In case of problems, you can delete the kernel using the following steps:
1. Look up the kernel name by executing the command: jupyter kernelspec list
2. Delete the kernel: jupyter kernelspec uninstall kernel_name

## Load Data

Before we can start analyzing the data, we first have to load data from the corresponding csv files into an appropriate database schema. This is fake data.

In [1]:
PRAGMA foreign_keys = OFF;

DROP TABLE IF EXISTS purchases;
DROP TABLE IF EXISTS nutritionalValues;
DROP TABLE IF EXISTS livingIn;
DROP TABLE IF EXISTS households;
DROP TABLE IF EXISTS citizens;
DROP TABLE IF EXISTS articles;

PRAGMA foreign_keys = ON;

CREATE TABLE households (
    id INTEGER PRIMARY KEY,
    street TEXT,
    postcode INTEGER,
    city TEXT,
    floor INTEGER
);

CREATE TABLE citizens (
    id INTEGER PRIMARY KEY,
    firstname TEXT,
    lastname TEXT,
    birthday TEXT
);

CREATE TABLE livingIn (
    household_id INTEGER,
    citizen_id INTEGER,
    start TEXT,
    until TEXT,
    FOREIGN KEY(household_id) REFERENCES households(id),
    FOREIGN KEY(citizen_id) REFERENCES citizens(id),
    PRIMARY KEY(citizen_id, start, until)
);

CREATE TABLE articles (
    id INTEGER PRIMARY KEY,
    label TEXT,
    unit INTEGER
);

CREATE TABLE nutritionalValues (
    id INTEGER PRIMARY KEY,
    calories INTEGER,
    FOREIGN KEY(id) REFERENCES articles(id)
);

CREATE TABLE purchases (
    article_id INTEGER,
    citizen_id INTEGER,
    date TEXT,
    amount REAL,
    FOREIGN KEY(article_id) REFERENCES articles(id),
    FOREIGN KEY(citizen_id) REFERENCES citizens(id),
    PRIMARY KEY(article_id, citizen_id, date)
);

In [2]:
-- enable csv mode:
.mode csv

-- import the necessary files:
.import data/nsa/households_no_header.csv households
.import data/nsa/citizens_no_header.csv citizens
.import data/nsa/livingIn_no_header.csv livingIn
.import data/nsa/articles_no_header.csv articles
.import data/nsa/nutritionalValues_no_header.csv nutritionalValues
.import data/nsa/purchases_no_header.csv purchases

-- enable pretty formatting:
.mode columns
.headers on

### Query 1

First we would like to identify the households that have a house number of 13 and in which at least 2 citizens live or lived that were born before 1920. Note that the house number is contained at the end of the `street` attribute of `households`. The output should contain the following attributes and should be ordered ascending by the street name:
1. The street of the household as "Straße",
2. The zip code of the household as "PLZ",
3. The city of the houshold as "Stadt".

In [3]:
SELECT street AS Straße, postcode AS PLZ, city AS Stadt
FROM households AS h
    JOIN livingIn
        ON h.id = household_id
    JOIN citizens AS c
        ON citizen_id = c.id
WHERE street LIKE '%13' AND birthday < '1920'
GROUP BY h.id
HAVING count(*) >= 2
ORDER BY Straße ASC;

Straße                PLZ    Stadt      
--------------------  -----  -----------
Bodenschatzstraße 13  66111  Saarbrücken
Korbinianstraße 13    66111  Saarbrücken
Leisringstraße 13     66111  Saarbrücken


In [4]:
DROP VIEW IF EXISTS q1_student;
CREATE VIEW q1_student AS
    SELECT street AS Straße, postcode AS PLZ, city AS Stadt
    FROM households AS h
        JOIN livingIn
            ON h.id = household_id
        JOIN citizens AS c
            ON citizen_id = c.id
    WHERE street LIKE '%13' AND birthday < '1920'
    GROUP BY h.id
    HAVING count(*) >= 2
    ORDER BY Straße ASC;

In [5]:
-- TEST
-- Prepare the necessary table for result comparison and load the data
-- You need to execute this cell only once
-- Repeated execution will not affect test results, but lead to error messages as you try to import the same data multiple times

DROP TABLE IF EXISTS q1;
CREATE TABLE q1 (
    Straße TEXT,
    PLZ INTEGER,
    Stadt TEXT,
    PRIMARY KEY(Straße, PLZ, Stadt)
);

-- import query results
.mode csv
.import data/nsa/tests/q1_no_header.csv q1

In [6]:
-- TEST
-- Note that this test compares the resulting tuples and does not ensure that your query is semantically correct.

-- compare query results
.mode columns

SELECT *
FROM (SELECT * FROM q1
      EXCEPT
      SELECT * FROM q1_student)
UNION
SELECT *
FROM (SELECT * FROM q1_student
      EXCEPT
      SELECT * FROM q1);
-- We expect an empty result.

Straße                PLZ               Stadt      
--------------------  ----------------  -----------
Apple                 511.17            265808.4   
Bean                  557.82            139455.0   
Beef                  530.32            1124278.4  
Beer                  577.889999999999  248492.7   
Beetroot              568.87            244614.1   
Bodenschatzstraße 13  66111             Saarbrücken
Buttermilk            519.93            197573.4   
Carrot                535.25            192690.0   
Corn                  493.13            532580.4   
Cream                 565.83            1154293.2  
Egg                   555.4             860870.0   
Emmentaler            514.83            1966650.6  
Grape                 610.65            427455.0   
Ham                   544.51            1824108.5  
Herring               564.47            824126.2   
Kohlrabi              560.43            151316.1   
Korbinianstraße 13    66111             Saarbrücken
Leisringstra

### Query 2

Now, we would like to find for each article, which citizen(s) bought the most amount of it in a single purchase. Your output should contain the following attributes and should be ordered descending by the label of the article:

1. The label of the article as "Bezeichnung",
2. The maximum amount purchased in a single purchase of that particular article (either in liter or kilogram) as "Menge",
3. The first name of the citizen as "Vorname",
4. The last name of the citizen as "Nachname"

In [7]:
SELECT label AS Bezeichnung, max_amount AS Menge, firstname AS Vorname, lastname AS Nachname
FROM (
    SELECT article_id, max(amount) AS max_amount
    FROM purchases 
    GROUP BY article_id
    ) AS p1
    JOIN purchases AS p2
        ON p1.article_id = p2.article_id AND amount = max_amount
    JOIN citizens AS c
        ON citizen_id = c.id
    JOIN articles AS a
        ON p1.article_id = a.id
ORDER BY Bezeichnung DESC
LIMIT 10;

Bezeichnung   Menge  Vorname    Nachname
------------  -----  ---------  --------
Wine          3.94   Marie      Huber   
Veal          5.12   Pia        Fuchs   
Trout         4.89   Sebastian  Fuchs   
Strawberry    4.88   Gottfried  Fuchs   
Salad         4.3    Sebastian  Fuchs   
Red Cabbage   4.93   Hilde      Fuchs   
Raspberry     5.09   Pia        Fuchs   
Pumpkin       4.59   Hilde      Fuchs   
Pumpernickel  4.25   Pia        Fuchs   
Pretzel       4.52   Pia        Fuchs   


In [8]:
DROP VIEW IF EXISTS q2_student;
CREATE VIEW q2_student AS
    SELECT label AS Bezeichnung, max_amount AS Menge, firstname AS Vorname, lastname AS Nachname
    FROM (
        SELECT article_id, max(amount) AS max_amount
        FROM purchases 
        GROUP BY article_id
        ) AS p1
        JOIN purchases AS p2
            ON p1.article_id = p2.article_id AND amount = max_amount
        JOIN citizens AS c
            ON citizen_id = c.id
        JOIN articles AS a
            ON p1.article_id = a.id
    ORDER BY Bezeichnung DESC
    LIMIT 10;

In [9]:
-- TEST
-- Prepare the necessary table for result comparison and load the data
-- You need to execute this cell only once
-- Repeated execution will not affect test results, but lead to error messages as you try to import the same data multiple times

DROP TABLE IF EXISTS q2;
CREATE TABLE q2 (
    Bezeichnung TEXT,
    Menge INTEGER,
    Vorname TEXT,
    Nachname TEXT,
    PRIMARY KEY(Bezeichnung)
);

-- import query results
.mode csv
.import data/nsa/tests/q2_no_header.csv q2

In [10]:
-- TEST
-- Note that this test compares the resulting tuples and does not ensure that your query is semantically correct.

-- compare query results
.mode columns

SELECT *
FROM (SELECT q2.Bezeichnung, ROUND(q2.Menge, 5) AS Menge, q2.Vorname, q2.Nachname FROM q2
      EXCEPT
      SELECT q2_student.Bezeichnung, ROUND(q2_student.Menge, 5) AS Menge, q2_student.Vorname, q2_student.Nachname FROM q2_student)
UNION
SELECT *
FROM (SELECT q2_student.Bezeichnung, ROUND(q2_student.Menge, 5) AS Menge, q2_student.Vorname, q2_student.Nachname FROM q2_student
      EXCEPT
      SELECT q2.Bezeichnung, ROUND(q2.Menge, 5) AS Menge, q2.Vorname, q2.Nachname FROM q2);
-- We expect an empty result.

Bezeichnung   Menge  Vorname    Nachname        
------------  -----  ---------  ----------------
Anna          0.0    9          878759.799999999
Charlotte     0.0    5          892734.3        
Emil          0.0    5          809190.3        
Frieda        0.0    11         677714.2        
Fritz         0.0    7          1303957.5       
Gerhard       0.0    9          538973.0        
Gottfried     0.0    13         680694.2        
Helene        0.0    2          387827.9        
Herbert       0.0    1          439283.2        
Hilde         0.0    13         599845.8        
Johann        0.0    10         426283.6        
Karl          0.0    6          438986.4        
Liselotte     0.0    14         462413.0        
Marie         0.0    8          957446.499999999
Markus        0.0    4          431704.0        
Maximilian    0.0    15         563572.9        
Oskar         0.0    3          324718.2        
Pauline       0.0    4          446149.8        
Peter         0.0   

### Query 3

Lastly, we want to identify the citizens that moved in two households between 1900 and 1949 (both included). You should also compute the purchased calories for each of these citizens during the time period between 1900 and 1949. Note, that the `calories` in `nutritionalValues` are given in kcal/100g, while the `amount` of a bought article in `purchases` is given in kg. Therefore you need to convert kcal/100g in kcal/kg. The output should consist of the following attributes, ordered descending by "Kalorien".
1. The citizen's first name as "Vorname",
2. The citizen's last name as "Nachname",
3. The total amount of calories purchased during that time period by the citizen as "Kalorien".

In [11]:
SELECT firstname AS Vorname, lastname AS Nachname, sum(sum_calories_each_article) AS Kalorien
FROM (
    SELECT l.citizen_id, sum(amount*calories*1000) AS sum_calories_each_article
    FROM (
        SELECT citizen_id
        FROM livingIn
        WHERE '1900' <= start <= '1949'
        GROUP BY citizen_id
        HAVING count(*) = 2
        ) AS l
        JOIN purchases AS p
            ON l.citizen_id = p.citizen_id
        JOIN nutritionalValues AS n
            ON n.id = article_id
    WHERE '1900' <= date <= '1949'
    GROUP BY l.citizen_id, article_id) 
    JOIN citizens
        ON id = citizen_id
GROUP BY id
ORDER BY Kalorien DESC;

Vorname  Nachname   Kalorien   
-------  ---------  -----------
Fritz    Wolf       130395750.0
Theresa  Herrmann   79388580.0 
Theodor  Herrmann   51518750.0 
Pauline  Schneider  44614980.0 
Markus   Schneider  43170400.0 


In [12]:
DROP VIEW IF EXISTS q3_student;
CREATE VIEW q3_student AS
    SELECT firstname AS Vorname, lastname AS Nachname, sum(sum_calories_each_article) AS Kalorien
    FROM (
        SELECT l.citizen_id, sum(amount*calories*1000) AS sum_calories_each_article
        FROM (
            SELECT citizen_id
            FROM livingIn
            WHERE '1900' <= start <= '1949'
            GROUP BY citizen_id
            HAVING count(*) = 2
            ) AS l
            JOIN purchases AS p
                ON l.citizen_id = p.citizen_id
            JOIN nutritionalValues AS n
                ON n.id = article_id
        WHERE '1900' <= date <= '1949'
        GROUP BY l.citizen_id, article_id) 
        JOIN citizens
            ON id = citizen_id
    GROUP BY id;

In [13]:
-- TEST
-- Prepare the necessary table for result comparison and load the data
-- You need to execute this cell only once
-- Repeated execution will not affect test results, but lead to error messages as you try to import the same data multiple times

DROP TABLE IF EXISTS q3;
CREATE TABLE q3 (
    Vorname TEXT,
    Nachname TEXT,
    Kalorien REAL,
    PRIMARY KEY(Vorname, Nachname)
);

-- import query results
.mode csv
.import data/nsa/tests/q3_no_header.csv q3

data/nsa/tests/q3_no_header.csv:1: expected 3 columns but found 4 - extras ignored
data/nsa/tests/q3_no_header.csv:2: expected 3 columns but found 4 - extras ignored
data/nsa/tests/q3_no_header.csv:3: expected 3 columns but found 4 - extras ignored
data/nsa/tests/q3_no_header.csv:4: expected 3 columns but found 4 - extras ignored


In [14]:
-- TEST
-- Note that this test compares the resulting tuples and does not ensure that your query is semantically correct.

-- compare query results
.mode columns

SELECT *
FROM (SELECT q3.Vorname, q3.Nachname, ROUND(q3.Kalorien, 5) AS Kalorien FROM q3
      EXCEPT
      SELECT q3_student.Vorname, q3_student.Nachname, ROUND(q3_student.Kalorien, 5) AS Kalorien FROM q3_student)
UNION
SELECT *
FROM (SELECT q3_student.Vorname, q3_student.Nachname, ROUND(q3_student.Kalorien, 5) AS Kalorien FROM q3_student
      EXCEPT
      SELECT q3.Vorname, q3.Nachname, ROUND(q3.Kalorien, 5) AS Kalorien FROM q3);
-- We expect an empty result.

Vorname  Nachname             Kalorien   
-------  -------------------  -----------
13       Bergmannweg 7        66111.0    
2        Königinstraße 25     66111.0    
4        Wiesenheimstraße 26  66111.0    
8        Passusgasse 7        66111.0    
Fritz    Wolf                 130395750.0
Markus   Schneider            43170400.0 
Pauline  Schneider            44614980.0 
Theodor  Herrmann             51518750.0 
Theresa  Herrmann             79388580.0 
